## DV2599 Assignment 2
Group 8


***
Viktor Fransson

vifr22@student.bth.se

***

Tobias Gustafsson

togu22@student.bth.se
***

#### Initialize

In [1]:
# Import packages
import pandas as pd
import statistics as stat
import numpy as np
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Load dataset
df = pd.read_csv('spambase/spambase.data', delimiter=',', header=None)

# Add headers to columns to increase clarity and comprehensibility
headers = []
with open('spambase/spambase.names', 'r') as file:
    N = [line for line in file]
    headers = [x[:x.find(':')] for x in N[33:]]

headers.append("spam")  # Spam column is last, 1 = spam, 0 = not spam
df.columns = headers

# Split data into x and y
y = df['spam']
x = df.drop(columns='spam')

display(df)

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


#### 0. Pre-process

In [2]:
# Initialize standard scaler
scaler = StandardScaler()

# Fit and transform train set
x_scaler = scaler.fit_transform(x)

# Turn back into pandas dataframe
x_scaled = pd.DataFrame(x_scaler, columns=x.columns, index=x.index)

# Describe
x_scaled.describe().round(2)


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
count,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,...,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00,4601.00
mean,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,...,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-0.34,-0.17,-0.56,-0.05,-0.46,-0.35,-0.29,-0.26,-0.32,-0.37,...,-0.11,-0.16,-0.51,-0.16,-0.33,-0.31,-0.10,-0.13,-0.26,-0.47
25%,-0.34,-0.17,-0.56,-0.05,-0.46,-0.35,-0.29,-0.26,-0.32,-0.37,...,-0.11,-0.16,-0.51,-0.16,-0.33,-0.31,-0.10,-0.11,-0.24,-0.41
50%,-0.34,-0.17,-0.56,-0.05,-0.46,-0.35,-0.29,-0.26,-0.32,-0.37,...,-0.11,-0.16,-0.27,-0.16,-0.33,-0.31,-0.10,-0.09,-0.19,-0.31
75%,-0.34,-0.17,0.28,-0.05,0.10,-0.35,-0.29,-0.26,-0.32,-0.12,...,-0.11,-0.16,0.18,-0.16,0.06,-0.10,-0.10,-0.05,-0.05,-0.03
max,14.53,10.90,9.56,30.64,14.41,21.13,18.28,27.44,18.56,27.83,...,34.89,17.85,35.56,37.15,39.49,24.11,46.09,34.59,50.99,25.66


#### 1. Stratified ten-fold cross-validation

In [3]:
# Initialize ten-fold cross-validation
skf = StratifiedKFold(n_splits=10)
score_types = ('fit_time', 'accuracy_score', 'f_score')

# Initialize models
rf = RandomForestClassifier()
lr = LogisticRegression()
nb = GaussianNB()
sv = SVC()

models = [("Random Forest", rf), ("Logistic Regression", lr), ("SVC", sv)]
model_data = defaultdict(dict)

# Stratified K-fold for each model
for model_name, model in models:
    cv_accuracy = cross_validate(model, x_scaled, y, cv=skf, scoring="accuracy")
    cv_f1 = cross_validate(model, x_scaled, y, cv=skf, scoring="f1")
    model_data[model_name][score_types[0]] = cv_accuracy.get('fit_time')
    model_data[model_name][score_types[1]] = cv_accuracy.get('test_score')
    model_data[model_name][score_types[2]] = cv_f1.get('test_score')
    # model_data[model_name][score_types[0]] = [0.5 for _ in range(10)]
    # model_data[model_name][score_types[1]] = [0.6 for _ in range(10)]
    # model_data[model_name][score_types[2]] = [0.2 for _ in range(10)]

#### 2. Results

In [4]:
column_1 = {"Fold":[1,2,3,4,5,6,7,8,9,10,"avg", "stdev"]}

### Create accuracy, f1 and time dataframes
df_acc = pd.DataFrame(column_1)
df_f1 = pd.DataFrame(column_1)
df_time = pd.DataFrame(column_1)

### Add data to dataframes
for model_name in model_data:
    for score_id, df in zip(score_types, (df_time, df_acc, df_f1)):
        score = model_data[model_name][score_id]
        df[model_name] = np.append(score, [stat.mean(score), stat.stdev(score)])

### Save tables
df_time.round(4).to_csv(f"df_time.csv", index=False)
df_acc.round(4).to_csv(f"df_acc.csv", index=False)
df_f1.round(4).to_csv(f"df_f.csv", index=False)

### Display
print("Time")
display(df_time)

print("Accuracy")
display(df_acc)

print("F1")
display(df_f1)

Time


,Fold,Random Forest,Logistic Regression,SVC
0,1,0.558894,0.016725,0.149526
1,2,0.605749,0.015001,0.142514
2,3,0.556413,0.013336,0.147538
3,4,0.536747,0.014000,0.144140
4,5,0.541345,0.013999,0.142520
5,6,0.567816,0.015003,0.148478
6,7,0.524469,0.013509,0.172208
7,8,0.525723,0.014000,0.148905
8,9,0.528560,0.015001,0.141043
9,10,0.528319,0.012999,0.137512


Accuracy


,Fold,Random Forest,Logistic Regression,SVC
0,1,0.954447,0.930586,0.924078
1,2,0.952174,0.921739,0.941304
2,3,0.934783,0.919565,0.934783
3,4,0.950000,0.939130,0.936957
4,5,0.963043,0.923913,0.947826
5,6,0.958696,0.930435,0.952174
6,7,0.969565,0.952174,0.956522
7,8,0.971739,0.926087,0.941304
8,9,0.884783,0.854348,0.891304
9,10,0.863043,0.871739,0.869565


F1


,Fold,Random Forest,Logistic Regression,SVC
0,1,0.940845,0.908046,0.902507
1,2,0.932961,0.897727,0.925208
2,3,0.922190,0.896359,0.915730
3,4,0.939828,0.922222,0.917847
4,5,0.952381,0.902507,0.932584
5,6,0.948229,0.914439,0.939891
6,7,0.960674,0.936782,0.941860
7,8,0.960894,0.904494,0.923513
8,9,0.879581,0.820375,0.861111
9,10,0.824513,0.829971,0.821429


#### 3. Friedman test

In [5]:
friedman_scores = {}

### Perform Friedman test for each dataframe
for i, df in enumerate((df_time, df_acc, df_f1)):
    ### Remove last two rows and fold column
    df_friedman_score = df.iloc[:-2]
    df_rank = df_friedman_score.drop(columns='Fold')

    ### Rank
    df_rank = df_rank.rank(1, 'max', ascending=not i)

    ### Combine ranks and score values
    df_friedman_score = df_friedman_score.rename(columns={'Fold': 'Data set'})
    for column in df_rank.columns:
        combines = [f"{round(x,4)} ({int(y)})" for x, y in zip(df_friedman_score[column], df_rank[column])]
        df_friedman_score[column] = combines

    ### Add avg rank row
    df_rank_avg = df_rank.mean()
    df_friedman_score.loc[len(df_friedman_score)] = ['avg rank', *[df_rank_avg.iloc[x] for x in range(len(df_rank_avg))]] 

    ### Save friedman table
    df_friedman_score.to_csv(f"friedman_{score_types[i]}.csv", index=False)
    friedman_scores[score_types[i]] = df_friedman_score

    print(f"{score_types[i]}:")
    display(df_friedman_score)

fit_time:


,Data set,Random Forest,Logistic Regression,SVC
0,1,0.5589 (3),0.0167 (1),0.1495 (2)
1,2,0.6057 (3),0.015 (1),0.1425 (2)
2,3,0.5564 (3),0.0133 (1),0.1475 (2)
3,4,0.5367 (3),0.014 (1),0.1441 (2)
4,5,0.5413 (3),0.014 (1),0.1425 (2)
5,6,0.5678 (3),0.015 (1),0.1485 (2)
6,7,0.5245 (3),0.0135 (1),0.1722 (2)
7,8,0.5257 (3),0.014 (1),0.1489 (2)
8,9,0.5286 (3),0.015 (1),0.141 (2)
9,10,0.5283 (3),0.013 (1),0.1375 (2)


accuracy_score:


,Data set,Random Forest,Logistic Regression,SVC
0,1,0.9544 (1),0.9306 (2),0.9241 (3)
1,2,0.9522 (1),0.9217 (3),0.9413 (2)
2,3,0.9348 (2),0.9196 (3),0.9348 (2)
3,4,0.95 (1),0.9391 (2),0.937 (3)
4,5,0.963 (1),0.9239 (3),0.9478 (2)
5,6,0.9587 (1),0.9304 (3),0.9522 (2)
6,7,0.9696 (1),0.9522 (3),0.9565 (2)
7,8,0.9717 (1),0.9261 (3),0.9413 (2)
8,9,0.8848 (2),0.8543 (3),0.8913 (1)
9,10,0.863 (3),0.8717 (1),0.8696 (2)


f_score:


,Data set,Random Forest,Logistic Regression,SVC
0,1,0.9408 (1),0.908 (2),0.9025 (3)
1,2,0.933 (1),0.8977 (3),0.9252 (2)
2,3,0.9222 (1),0.8964 (3),0.9157 (2)
3,4,0.9398 (1),0.9222 (2),0.9178 (3)
4,5,0.9524 (1),0.9025 (3),0.9326 (2)
5,6,0.9482 (1),0.9144 (3),0.9399 (2)
6,7,0.9607 (1),0.9368 (3),0.9419 (2)
7,8,0.9609 (1),0.9045 (3),0.9235 (2)
8,9,0.8796 (1),0.8204 (3),0.8611 (2)
9,10,0.8245 (2),0.83 (1),0.8214 (3)


#### 5.1 Friedman values

In [6]:
n = 10
k = len(models)
critical_value = 6.20
post_hoc = []

print(f"Critical value = {critical_value}")

### R_mean value
R_mean = (k+1)/2
print("R mean =", R_mean, "\n")

for i, df_name in enumerate(friedman_scores):
    df = friedman_scores.get(df_name)
    
    ### Sum of squared differences avg
    print(f"{df_name}:")
    avg = list(df.iloc[-1])[1:]
    
    sum1 = n*sum([(x-R_mean)**2 for x in avg])
    print(f"sum1 = {round(sum1,4)}")

    ### Sum of squared differences all
    sum2 = 0
    df_copy = df.iloc[:-1].drop(columns='Data set')
    for row in df_copy.to_numpy():
        for value in row:
            v = int(value[-2])
            sum2 += (v-R_mean)**2
    sum2 *= 1/(n*(k-1))
    
    ### Calculate ratio
    f_stat = sum1/sum2
    if f_stat > critical_value:
        post_hoc.append(df_name)

    print(f"sum2= {round(sum2,4)}")
    print(f"Friedman statistic = {round(f_stat,4)}")
    print()


Critical value = 6.2
R mean = 2.0 

fit_time:
sum1 = 20.0
sum2= 1.0
Friedman statistic = 20.0

accuracy_score:
sum1 = 7.3
sum2= 0.95
Friedman statistic = 7.6842

f_score:
sum1 = 12.6
sum2= 1.0
Friedman statistic = 12.6



##### Nemenyi test

In [7]:
### Critical distance
cd = round(2.343*(k*(k+1)/(6*n))**0.5, 4)
print(f"Critical distance = {cd}")
for i, df_name in enumerate(post_hoc):
    df = friedman_scores.get(df_name)
    print("----------")
    print(f"{df_name}:")

    ### Find names and averages
    avg = list(df.iloc[-1])[1:]
    cat = list(df)[1:]
    
    ### Compare differences with critical value
    for m_name, score in zip(cat, avg):
        for n_name, score2 in zip(cat, avg):
            if score-score2 > cd:
                print(f"[{m_name} - {n_name}] exceeds critical distance")



Critical distance = 1.0478
----------
fit_time:
[Random Forest - Logistic Regression] exceeds critical distance
----------
accuracy_score:
[Logistic Regression - Random Forest] exceeds critical distance
----------
f_score:
[Logistic Regression - Random Forest] exceeds critical distance
[SVC - Random Forest] exceeds critical distance
